In [1]:
!pip install flair -q

     |████████████████████████████████| 143kB 2.3MB/s 
     |████████████████████████████████| 983kB 51.8MB/s 
     |████████████████████████████████| 235kB 54.1MB/s 
     |████████████████████████████████| 798kB 46.3MB/s 
     |████████████████████████████████| 501kB 53.6MB/s 
     |████████████████████████████████| 870kB 46.7MB/s 
     |████████████████████████████████| 1.0MB 45.0MB/s 
     |████████████████████████████████| 3.7MB 32.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [2]:
!wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
!unzip -o drugsCom_raw.zip

--2020-03-08 14:24:30--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/x-httpd-php]
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip    100%[===================>]  41.00M  30.2MB/s    in 1.4s    

2020-03-08 14:24:32 (30.2 MB/s) - ‘drugsCom_raw.zip’ saved [42989872/42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [3]:
import pandas as pd

drugs = pd.read_csv('/content/drugsComTrain_raw.tsv', sep='\t', header=0)#, nrows=10000)
print(drugs.shape, drugs.columns)


(161297, 7) Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount'],
      dtype='object')


In [4]:
drugs[['Unnamed: 0','rating']].groupby('rating').count()

,Unnamed: 0
rating,
1.0,21619
2.0,6931
3.0,6513
4.0,5012
5.0,8013
6.0,6343
7.0,9456
8.0,18890
9.0,27531


In [5]:
drugs = pd.read_csv('/content/drugsComTrain_raw.tsv', sep='\t', header=0, nrows=20000)
drugs[['Unnamed: 0','rating']].groupby('rating').count()

,Unnamed: 0
rating,
1.0,2658
2.0,872
3.0,811
4.0,651
5.0,1019
6.0,777
7.0,1144
8.0,2269
9.0,3412


In [6]:
from flair.data import Sentence, Label

sentences = []
for idx in drugs.index:
  data = drugs.iloc[idx]
  sentence = Sentence(data.review, labels=[Label(value=str(data.rating))], use_tokenizer=True)
  sentences.append(sentence)

In [7]:
!wget -c https://ghc-flair.s3.amazonaws.com/PubMed-w2v.bin 

import gensim

word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/PubMed-w2v.bin', binary=True)
word_vectors.save('pubmed.model')

n_loop = 0

--2020-03-08 14:25:28--  https://ghc-flair.s3.amazonaws.com/PubMed-w2v.bin
Resolving ghc-flair.s3.amazonaws.com (ghc-flair.s3.amazonaws.com)... 52.216.24.212
Connecting to ghc-flair.s3.amazonaws.com (ghc-flair.s3.amazonaws.com)|52.216.24.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1909156308 (1.8G) [application/octet-stream]
Saving to: ‘PubMed-w2v.bin’

PubMed-w2v.bin      100%[===================>]   1.78G  75.4MB/s    in 24s     

2020-03-08 14:25:53 (74.9 MB/s) - ‘PubMed-w2v.bin’ saved [1909156308/1909156308]



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
from flair.embeddings import WordEmbeddings
custom_embedding =  WordEmbeddings('pubmed.model')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
from flair.data import Sentence, Corpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models.text_regression_model import TextRegressor
from flair.trainers import ModelTrainer


target_set = drugs['rating'].values

results = []

## TEST
kfold_test = StratifiedKFold(n_splits=3, random_state=13)
for i, (train, test) in enumerate(kfold_test.split(target_set, target_set)):

  if i > n_loop:
    break;

  if i < n_loop:
    continue;


  print(i, len(train), len(test))
  corpus = Corpus([sentences[t] for t in train], [sentences[d] for d in test], [sentences[e] for e in test]) 

  label_dict = corpus.make_label_dictionary()
  
  word_embeddings = [custom_embedding]
  
  document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                       hidden_size=512,
                                                                       reproject_words=True,
                                                                       reproject_words_dimension=256,
                                                                       )
  model = TextRegressor(document_embeddings)

  trainer = ModelTrainer(model, corpus)

  result = trainer.train('train_'+str(i)+'/')

  print(str(i) + " - Test score: " + str(result["test_score"]))


0 13333 6667
2020-03-08 14:26:50,085 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
100%|██████████| 13333/13333 [00:00<00:00, 236115.99it/s]

2020-03-08 14:26:50,170 [b'4.0', b'2.0', b'3.0', b'10.0', b'6.0', b'5.0', b'1.0', b'9.0', b'8.0', b'7.0']


2020-03-08 14:26:58,126 Using REGRESSION - experimental
2020-03-08 14:26:58,128 ----------------------------------------------------------------------------------------------------
2020-03-08 14:26:58,129 Model: "TextRegressor(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('pubmed.model')
    )
    (word_reprojection_map): Linear(in_features=200, out_features=256, bias=True)
    (rnn): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=1, bias=True)
  (loss_function): MSELoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-03-08 14:26:58,131 ----------------------------------------------------------------------------------------------------
2020-03-08 14:26:58,132 Corpus: "Corpus: 13333 train + 6667 dev + 6667 test sentences"
2020-03-08 14:26:58,134 ---------------------------------------------------------------

In [10]:
#!wget -c https://ghc-flair.s3.amazonaws.com/BioWordVec_PubMed_MIMICIII_d200.vec.bin 

word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/BioWordVec_PubMed_MIMICIII_d200.vec.bin', binary=True)
word_vectors.save('biowordvec.model')

embedding =  WordEmbeddings('biowordvec.model')



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [0]:
#!wget -c https://ghc-flair.s3.amazonaws.com/wiki.en.zip
!unzip wiki.en.zip

import gensim

word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/wiki.en.vec', binary=False)
word_vectors.save('wiki.model')

wiki_embedding =  WordEmbeddings('wiki.model')
